In [1]:
!pip list

Package                  Version
------------------------ ------------
accelerate               1.12.0
aiohappyeyeballs         2.6.1
aiohttp                  3.13.2
aiosignal                1.4.0
anyio                    4.12.0
asttokens                3.0.1
attrs                    25.4.0
bitsandbytes             0.48.2
certifi                  2025.11.12
charset-normalizer       3.4.4
comm                     0.2.3
cut-cross-entropy        25.1.1
datasets                 4.3.0
debugpy                  1.8.17
decorator                5.2.1
dill                     0.4.0
docstring_parser         0.17.0
executing                2.2.1
filelock                 3.20.0
frozenlist               1.8.0
fsspec                   2025.9.0
h11                      0.16.0
hf_transfer              0.1.9
hf-xet                   1.2.0
httpcore                 1.0.9
httpx                    0.28.1
huggingface-hub          0.36.0
idna                     3.11
ipykernel                7.1.0
ipython    

### Unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

model_name = "models/Qwen3-14B"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, # We have full finetuning now!
    # token = "hf_...",      # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/mnt/data2/icd10/venv_sft/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/mnt/data2/icd10/venv_sft/lib/python3.11/site-packages/unsloth/models/rl_replacements.py:946: UserWarning: You are importing from 'trl.experimental'. APIs here are unstable and may change or be removed without notice. Silence this warning by setting environment variable TRL_EXPERIMENTAL_SILENCE=1.
  import trl.experimental.openenv.utils as openenv_utils
[unsloth_zoo.log|WARNING]Unsloth: Failed to import trl openenv: No module named 'trl.experimental.openenv'


==((====))==  Unsloth 2025.12.1: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    NVIDIA RTX 6000 Ada Generation. Num GPUs = 2. Max memory: 47.493 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 8/8 [00:21<00:00,  2.67s/it]


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,   # We support rank stabilized LoRA
    loftq_config = None,  # And LoftQ
)

Unsloth 2025.12.1 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


### Data Prep

In [4]:
from datasets import load_dataset

# 1. 讀取您的 CSV 檔案
# 假設您的 CSV 檔案就在當前目錄下，名為 "my_data.csv"
dataset = load_dataset("csv", data_files="0.6_acc_result.csv", split="train")

# 2. 修改轉換函數，對應您的 CSV 欄位名稱
def generate_conversation(examples):
    # 修改這裡：把 CSV 裡的 "question" 欄位抓出來
    problems  = examples["question"]
    
    # 修改這裡：把 CSV 裡的 "assistant_reply" 欄位抓出來
    solutions = examples["assistant_reply"]
    
    conversations = []
    # 使用 zip 將問題和回答配對
    for problem, solution in zip(problems, solutions):
        conversations.append([
            {"role" : "user",      "content" : problem},
            {"role" : "assistant", "content" : solution},
        ])
    
    return { "conversations": conversations }

# 3. 執行轉換
# 這會產生一個新的 "conversations" 欄位，裡面裝好格式化的對話
dataset = dataset.map(generate_conversation, batched=True)

print(dataset[0]["conversations"])

Map: 100%|██████████| 213/213 [00:00<00:00, 18688.93 examples/s]

[{'content': '患者 20000024 的病歷如下：Sex:   F\n \nService: MEDICINE\n \nAllergies: \nAspirin\n \nAttending: ___.\n \nChief Complaint:\nweakness, diarrhea\n \nMajor Surgical or Invasive Procedure:\nNone\n\n \nHistory of Present Illness:\nMs. ___ is a ___ year-old woman with PMH significant for \nchronic anemia, osteoporosis, hypertension, ataxia, and recent \nL5 fracture in the setting of recurrent falls who presents from \nhome with fatigue and generalized weakness and diarrhea.  \n The patient\'s recent history is notable for the follow:  \n - On ___, she presented with 4 days of LBP s/p fall from \nstanding at which time imaging revealed acute L5 fracture. She \nwas evaluated by Spine team who recommended early mobilization, \npain control, but no brace required. She was evaluated by ___, \nand discharged to ___.  \n - She was discharged home with ___ on ___.  \n - On ___, she again presented to ___ s/p fall from \nstanding while trying to reach for a glass of water. She did \nhave a occi

In [5]:
# 這裡確保使用正確的 Chat Template (例如 Qwen, Llama 等)
from unsloth.chat_templates import get_chat_template

# 確保 tokenizer 知道要用哪種格式 (這行通常在載入模型時會做，但保險起見可以加)
tokenizer = get_chat_template(tokenizer, chat_template = "qwen-3") 

SYSTEM_PROMPT = """你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
你將會得知病人的ICD-10代碼跟病例並回推為什麼病人的ICD-10是如此：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結病人的ICD-10編碼集合。  
</reasoning>  
<answer>  
僅輸出正確的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
"""
def formatting_prompts_func(examples):
    conversations = examples["conversations"]
    texts = []
    
    for convo in conversations:
        convo.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        text = tokenizer.apply_chat_template(
                convo, 
                tokenize = False, 
                add_generation_prompt = False
            )
        texts.append(text)
    return { "text" : texts, }

# 執行轉換：這會產生一個新的 "text" 欄位
dataset = dataset.map(formatting_prompts_func, batched = True)

# 檢查結果：現在模型真正看到的是這個樣子
print(dataset[0]["text"])

Map: 100%|██████████| 213/213 [00:00<00:00, 3951.45 examples/s]

<|im_start|>system
你是一名專門根據患者用藥與病史紀錄預測 ICD-10 編碼的醫學專家。  
你將會得知病人的ICD-10代碼跟病例並回推為什麼病人的ICD-10是如此：  

<reasoning>  
逐步推理：逐條說明病例中每個臨床資訊（症狀、診斷、檢查結果、既往病史、用藥等）可能對應的 ICD-10 編碼，並在最後總結病人的ICD-10編碼集合。  
</reasoning>  
<answer>  
僅輸出正確的 ICD-10 編碼，使用英文逗號分隔，不要添加任何額外文字或解釋。  
</answer>  
<|im_end|>
<|im_start|>user
患者 20000024 的病歷如下：Sex:   F
 
Service: MEDICINE
 
Allergies: 
Aspirin
 
Attending: ___.
 
Chief Complaint:
weakness, diarrhea
 
Major Surgical or Invasive Procedure:
None

 
History of Present Illness:
Ms. ___ is a ___ year-old woman with PMH significant for 
chronic anemia, osteoporosis, hypertension, ataxia, and recent 
L5 fracture in the setting of recurrent falls who presents from 
home with fatigue and generalized weakness and diarrhea.  
 The patient's recent history is notable for the follow:  
 - On ___, she presented with 4 days of LBP s/p fall from 
standing at which time imaging revealed acute L5 fracture. She 
was evaluated by Spine team who recommended early mobilization, 
pain c

### Train the model

In [6]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 8,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 10,
        num_train_epochs = 4, # Set this for 1 full training run.
        # max_steps = 50,
        learning_rate = 2e-4, # Reduce to 2ez-5 for long training runs
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16): 100%|██████████| 213/213 [00:03<00:00, 70.10 examples/s] 


In [7]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 213 | Num Epochs = 4 | Total steps = 56
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 128,450,560 of 14,896,757,760 (0.86% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.845200
2,1.740000
3,1.757200
4,1.736600
5,1.788200
6,1.733100
7,1.616800
8,1.553100
9,1.544100
10,1.517300


In [8]:
save_path = f"{model_name}-sft_v2"

# lora 方式儲存
model.save_pretrained_merged(save_path, tokenizer, save_method="merged_16bit") 

# # 直接儲存模型與 tokenizer
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

Detected local model directory: /mnt/data2/icd10/models/Qwen3-14B
Found HuggingFace hub cache directory: /home/justin/.cache/huggingface/hub


Unsloth: Preparing safetensor model files:  12%|█▎        | 1/8 [00:01<00:13,  1.94s/it]

Copied model-00002-of-00008.safetensors from local model directory


Unsloth: Preparing safetensor model files:  25%|██▌       | 2/8 [00:03<00:11,  1.93s/it]

Copied model-00004-of-00008.safetensors from local model directory


...

Copied model-00001-of-00008.safetensors from local model directory


Unsloth: Preparing safetensor model files: 100%|██████████| 8/8 [00:14<00:00,  1.78s/it]


Copied model-00008-of-00008.safetensors from local model directory


Unsloth: Merging weights into 16bit: 100%|██████████| 8/8 [04:43<00:00, 35.44s/it]


Unsloth: Merge process complete. Saved to `/mnt/data2/icd10/models/Qwen3-14B-sft_v2`
